# Load data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
data = './processed/'
batch_size = 32
image_size=(256,256)
training_set = tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=0.5,
    subset="training",
    image_size=image_size,
    seed=123,
    batch_size=batch_size,
)

testing_set = tf.keras.utils.image_dataset_from_directory(
    data,
    validation_split=0.5,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size,
)


In [ ]:
training_set

In [ ]:
class_names = training_set.class_names
print(class_names)

In [ ]:
for images, labels in training_set:
    print(images.shape)
    print(labels.numpy())
    for image in images:
        plt.imshow(image.numpy().astype("uint8"))
        plt.show()
    break

Visualize the data

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in training_set.take(5):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in training_set:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", input_shape=(256, 256, 3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),]
)

In [ ]:
# test the data augmentation layer on a single image
for image, _ in training_set.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0])
        plt.axis("off")

In [ ]:
aug_ds=training_set.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
aug_ds=aug_ds.unbatch()

In [ ]:
for image_batch, labels_batch in aug_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


In [ ]:
aug_ds2=training_set.map(lambda x, y: (data_augmentation(x), y))

In [ ]:
aug_ds2=aug_ds2.unbatch()

In [ ]:
for i in aug_ds2:
    print(i[0].shape)

In [ ]:
aug_ds3=aug_ds.concatenate(aug_ds2)

In [ ]:
for i in aug_ds3:
    plt.imshow(i[0])
    plt.show()

In [ ]:
aug_ds3=aug_ds3.batch(32)

In [ ]:
for image_batch, labels_batch in aug_ds3:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
for images, labels in aug_ds3:
    print(images.shape)
    print(labels.numpy())
    for image in images:
        plt.imshow(image)
        plt.show()
    break


In [ ]:
# generating train and test data
number_of_augments=4
_aug_ds_new = training_set.unbatch()
aug_ds_new = training_set.shuffle(100).map(lambda x, y: (data_augmentation(x), y))

aug_ds_new = aug_ds_new.unbatch()
aug_ds_new = aug_ds_new.concatenate(_aug_ds_new)


In [ ]:
for images, labels in aug_ds_new:
    print(images.shape)
    print(labels.numpy())
        
    

In [ ]:

for i in range(number_of_augments):
    aug_ds_=training_set.shuffle(100).map(lambda x, y: (data_augmentation(x), y))
    print(i)
    for images, labels in aug_ds_:
        print(images.shape)
    
    print('-----------------')
    aug_ds_new=aug_ds_new.concatenate(aug_ds_.unbatch())
    for images, labels in aug_ds_new:
        print(images.shape)
    print('-----------------')


In [ ]:
for images, labels in aug_ds_new:
    print(images.shape)
    print(labels.numpy())

In [ ]:

aug_ds_new=aug_ds_new.batch(32)

In [ ]:
for image_batch, labels_batch in aug_ds_new:
  print(image_batch.shape)
  print(labels_batch.numpy())

In [ ]:
for images, labels in aug_ds_new:
    print(images.shape)
    print(labels.numpy())
    for image in images:
        plt.imshow(image.numpy().astype("uint8"))
        plt.show()
    break


In [ ]:
train_data_augmented = aug_ds_new

In [ ]:
#normalizing test data. Training data is already normalized
normalization_layer = tf.keras.layers.Rescaling(1./255)
test_data_normalized = testing_set.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
class_names = training_set.class_names
class_names

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])


In [ ]:
model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])



In [ ]:
epochs = 25
history = model.fit(
    train_data_augmented,
    validation_data=testing_set,
    epochs=epochs
)

In [ ]:
im_path = 'many.jpeg'

img = tf.keras.utils.load_img(
    im_path, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


In [ ]:
# pickle the model
import pickle
with open('model_khalil_malek.pkl', 'wb') as f:
    pickle.dump(model, f)


In [ ]:
model.save('model_khalil_malek.h5')

In [ ]:
# load the model
import tensorflow as tf
loaded_model = tf.keras.models.load_model('model9.h5')

In [ ]:
class_names = ['khalil', 'others']

In [ ]:
import numpy as np
im_path = 'others18.jpg'

img = tf.keras.utils.load_img(
    im_path, target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create a batch

predictions = loaded_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
